<a href="https://colab.research.google.com/github/squadOito/soulcodead2/blob/joseaureliok%2Fnotebook/notebooks/notebook_mongoDB_carregamento_parquet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Projeto Final**
Escola: SoulCode Academy

Curso: Bootcamp Analista de Dados - Martech - AD2

**Equipe 08**

**Alunos: Adriano Kim, José Aurelio, Marcos Paulo, Paulo Vitorino, Renato e Wesley**

Professores: Douglas Ribeiro, Franciane Rodrigues e Jonathas Carneiro

## Preparação de Ambiente

In [ ]:
# Instalação bibliotecas
!pip install -q pymongo
!pip install -q gcsfs

In [ ]:

# Importação de bibliotecas
import os
import pandas as pd
import numpy as np

from google.colab import data_table
from google.cloud import storage
from google.colab import drive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pymongo import MongoClient

### Função Chave

In [ ]:
def connectKey (chave):
  '''
  Conecta ao drive para obtenção de chave
  chave = nome da chave em string
  '''
  # Cria compartilhamento com Google Drive
  drive.mount('/content/drive', force_remount=True)

  # Arquivo a ser acessado na pasta compartilhada
  target = chave

  # Caminho completo da pasta compartilhada
  folder = '/content/drive/MyDrive/Classroom/AD2 - Analista de Dados/ProjetoFinal'

  # Acesso ao arquivo no colab
  return os.path.join(folder, target)



### Compartilhamento Chave GDrive

In [ ]:
# Nome da chave a ser acessada
chaveGcp = 'projeto-final-ad2-e8-ae566c3a2c2b.json'

# Chave GCP
serviceAccount = connectKey(chaveGcp)

Mounted at /content/drive


In [ ]:
# Nome da chave a ser acessada
chaveMongo = 'X509-cert-5608832275999674572.pem'

# Chave MongoDB
mongoKey = connectKey(chaveMongo)

Mounted at /content/drive


### Configuração Conexão MongoDB

In [ ]:
# configurando conexão
uri = "mongodb+srv://energy.drstakp.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,
					 tls=True,
					 tlsCertificateKeyFile= mongoKey)

### Configuração da bucket

In [ ]:
# Nome da bucket
bucket_name = 'projeto-final-ad2-e8'

# Caminho do prefixo
prefix_name = 'dados/backup/'

In [ ]:
# Conexão com a conta do Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# Conexão com a bucket do Google Cloud
clientGCP = storage.Client()
bucket = clientGCP.get_bucket(bucket_name)

### Conjuntos de dados

In [ ]:
# Formatação tabular
data_table.enable_dataframe_formatter()

#data_table.disable_dataframe_formatter()

In [ ]:
# Listando conjuntos de dados na bucket
blobList = [x.name for x in bucket.list_blobs(prefix = prefix_name)]

# dataframe lista blobs
blobDf = pd.DataFrame(blobList)

blobDf

,0
0,dados/backup/
1,dados/backup/2_1_cap_instalada_por_regiao_e_uf...
2,dados/backup/2_2_cap_instalada_de_geracao_elet...
3,dados/backup/2_3_geracao_eletrica_por_fonte_gw...
4,dados/backup/Veiculos_eletricos_brasil_anfavea...
5,dados/backup/Veiculos_eletricos_brasil_anfavea...
6,dados/backup/fator_capacidade_mwh_2015_2023.csv
7,dados/backup/global_horizontal_means.csv
8,dados/backup/iea_ponto_recarga_tratado.csv
9,dados/backup/iea_veiculos_tratado.csv


### Seleção do conjunto de dados

In [ ]:
# Seleção do arquivo parquet
listaDf = [blobDf[0][1],
           blobDf[0][2],
           blobDf[0][3],
           blobDf[0][4],
           blobDf[0][6],
           blobDf[0][7],
           blobDf[0][8],
           blobDf[0][10]
]

In [ ]:
listaDf

['dados/backup/2_1_cap_instalada_por_regiao_e_uf_tratado.xlsx',
 'dados/backup/2_2_cap_instalada_de_geracao_eletrica_por_fonte_mw.csv',
 'dados/backup/2_3_geracao_eletrica_por_fonte_gwh_tratado.xlsx',
 'dados/backup/Veiculos_eletricos_brasil_anfavea_2012_2023_crescimento.csv',
 'dados/backup/fator_capacidade_mwh_2015_2023.csv',
 'dados/backup/global_horizontal_means.csv',
 'dados/backup/iea_ponto_recarga_tratado.csv',
 'dados/backup/tratado_geracao_distribuida.csv']

In [ ]:
# Caminho gsutil do arquivo

pathC = {
        'instalada_fonte' : f'gs://{bucket_name}/{listaDf[1]}',
        'veiculos_anfavea' : f'gs://{bucket_name}/{listaDf[3]}',
        'fator_capacidade' : f'gs://{bucket_name}/{listaDf[4]}',
        'global_means' : f'gs://{bucket_name}/{listaDf[5]}',
        'iea_ponto_recarga' : f'gs://{bucket_name}/{listaDf[6]}',
        'geracao_distribuida' : f'gs://{bucket_name}/{listaDf[7]}'
}

pathE = {
        'instalada_regiao' : f'gs://{bucket_name}/{listaDf[0]}',
        'geracao_eletrica_fonte' : f'gs://{bucket_name}/{listaDf[2]}'
}

## Collection

In [ ]:
# Criando instância client
db = client['backup_tratados']

# Criando coleções
collection_csv = db['csv']
collection_excel = db['excel']

### Verificação

In [ ]:
# Verificando valores no MongoDB
doc_count_csv = collection_csv.count_documents({})
doc_count_excel = collection_excel.count_documents({})

print( f'\n-CSV: {doc_count_csv} \n-EXCEL: {doc_count_excel}')


-CSV: 0 
-EXCEL: 0


## Integração Google Cloud Storage - MongoDB

### Extração da bucket

#### CSV

In [ ]:
dictC = {}

for k, v in pathC.items():
  dfC = pd.read_parquet(v)

  dictC[k] = dfC

print('Done')

Done


#### EXCEL

In [ ]:
dictE = {}

for k, v in pathE.items():
  dfE = pd.read_parquet(v)

  dictE[k] = dfE

print('Done')

Done


### Carregamento na coleção

In [ ]:
# geracao_eletrica_fonte
collection_excel.insert_many(dictE['geracao_eletrica_fonte'].to_dict("records"))

print('Done')

Done


In [ ]:
# instalada_regiao
collection_excel.insert_many(dictE['instalada_regiao'].to_dict("records"))

print('Done')

Done


In [ ]:
# instalada_fonte
collection_csv.insert_many(dictC['instalada_fonte'].to_dict("records"))

print('Done')

Done


In [ ]:
# veiculos_anfavea
collection_csv.insert_many(dictC['veiculos_anfavea'].to_dict("records"))

print('Done')

Done


In [ ]:
# global_means
collection_csv.insert_many(dictC['global_means'].to_dict("records"))

print('Done')

Done


In [ ]:
# iea_ponto_recarga
collection_csv.insert_many(dictC['iea_ponto_recarga'].to_dict("records"))

print('Done')

Done


In [ ]:
# fator_capacidade
collection_csv.insert_many(dictC['fator_capacidade'].to_dict("records"))

print('Done')

Done


In [ ]:
# geracao_distribuida
collection_csv.insert_many(dictC['geracao_distribuida'].to_dict("records"))

print('Done')

Done


#### Verificação

In [ ]:
# Verificando valores no MongoDB
doc_count_csv = collection_csv.count_documents({})
doc_count_excel = collection_excel.count_documents({})

print( f'\n-CSV: {doc_count_csv} \n-EXCEL: {doc_count_excel}')


-CSV: 73014 
-EXCEL: 660


# Status

In [ ]:
print("BACKUP CONCLUÍDO COM SUCESSO!!!")

BACKUP CONCLUÍDO COM SUCESSO!!!
